<a href="https://colab.research.google.com/github/pachterlab/LSCHWCP_2023/blob/main/Notebooks/Supp_Fig_3/Supp_Fig_3d/1_macaque_celltype_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Host cell type assignment

In [ ]:
!pip install -q anndata gget

In [ ]:
import anndata
import numpy as np
import pandas as pd
import json
import os
import gget
import glob
import matplotlib.pyplot as plt
%config InlineBackend.figure_format='retina'
%load_ext blackcellmagic

import scanpy as sc
# sc.set_figure_params(figsize=(6, 6), frameon=False)
# sc.settings.n_jobs=2

def nd(arr):
    """
    Function to transform numpy matrix to nd array.
    """
    return np.asarray(arr).reshape(-1)

___
# Load data

The count matrix was generated in [this notebook](https://github.com/pachterlab/LSCHWCP_2023/blob/main/Notebooks/Supp_Fig_3/Supp_Fig_3abc/2_host_normalization_clustering.ipynb).

In [ ]:
# Load filtered, normalized and clustered macaque count matrix from Caltech Data
# !wget

In [ ]:
host_adata = anndata.read("macaque_QC_norm_leiden.h5ad")
host_adata

___
# Load marker genes

In [ ]:
!wget https://raw.githubusercontent.com/pachterlab/LSCHWCP_2023/main/Notebooks/Supp_Fig_3/Supp_Fig_3d/marker_genes.csv
marker_df = pd.read_csv("marker_genes.csv")

In [ ]:
# Get Ensembl IDs from gene symbols using gget
search_results = gget.search(marker_df["gene_symbol"].values, species="macaca_mulatta", release=109)
# Filter gget results for exact matches and protein-coding biotypes
search_results_filtered = search_results[search_results["gene_name"].isin(marker_df["gene_symbol"].values)][search_results["biotype"]=="protein_coding"]

In [ ]:
# Add Ensembl IDs to marker gene df
marker_df["ensembl_id"] = np.NaN

ensembl_id_list = []

for i, gene_symbol in enumerate(marker_df["gene_symbol"].values):
    ensembl_ids = search_results_filtered[
        search_results_filtered["gene_name"] == gene_symbol
    ]["ensembl_id"].values

    if len(ensembl_ids) < 1:
        if gene_symbol.startswith("ENS"):
            ensembl_id_list.append([gene_symbol])
        else:
            print(f"An Ensembl ID for gene {gene_symbol} could not be found.")

    else:
        ensembl_id_list.append(ensembl_ids)

marker_df["ensembl_id_list"] = ensembl_id_list
marker_df["ensembl_id"] = [id_list[0] for id_list in ensembl_id_list]
marker_df

___

In [ ]:
def flatten(xss):
    """
    Function to flatten array of arrays.
    """
    return [x for xs in xss for x in xs]

Add gene names of marker genes to adata:

In [ ]:
host_adata.var["gene_symbol"] = [np.NaN] * len(host_adata.var)

for ensembl_id, gene_symbol in zip(marker_df["ensembl_id"].values, marker_df["gene_name"].values):
    host_adata.var.loc[host_adata.var.index == ensembl_id, "gene_symbol"] = gene_symbol

In [ ]:
host_adata.var[~host_adata.var['gene_symbol'].isnull()]

Plot marker gene expression per group:

In [ ]:
celltypes = ['T cells', 'B cells', 'Plasmablast', 'Natural killer cells', 'Dendritic cells', 'Immature neutrophils', 'Monocytes', 'Panleukocyte', 'Cytokines', 'Proliferation']

# Grab markers in order of celltypes
markers = []
x_labels = []
for celltype in celltypes:
    markers = markers + list(marker_df.iloc[marker_df.index[marker_df['celltype'] == celltype].tolist()]["ensembl_id"].values)
    x_labels = x_labels + list(marker_df.iloc[marker_df.index[marker_df['celltype'] == celltype].tolist()]["gene_name"].values)

In [ ]:
var_groups = []
start = 0
for celltype in celltypes:
    c_indeces = marker_df.index[marker_df["celltype"] == celltype].tolist()
    var_groups.append((start, start+(c_indeces[-1] - c_indeces[0])))
    start = start+(c_indeces[-1] - c_indeces[0]) + 1

In [ ]:
fontsize = 14

temp_var_labels = [""] * len(celltypes)

ax = sc.pl.dotplot(
    host_adata,
    markers,
    groupby="leiden",
    use_raw=True,
    dendrogram=True,
    var_group_positions=var_groups,
    var_group_labels=temp_var_labels,
    var_group_rotation=45,
    swap_axes=False,
    standard_scale="var",
    show=False,
)

# ax["mainplot_ax"].set_title(f"Expression of marker genes in {'-'.join(adata.obs['tissue'][0].split('/'))} dataset", pad=120)

## Rotate var labels
# Get initial label positions from: ax["gene_group_ax"].properties()["children"]
var_positions = [2.95, 7.449999999999999, 9.45, 11.45, 13.95, 16.45, 22.45, 27.45, 29.95, 32.45]
for position, label in zip(var_positions, celltypes):
    ax["gene_group_ax"].text(position, 1.1, label, fontsize=fontsize, rotation=45, ha="left")

ax["mainplot_ax"].set_xticklabels(x_labels, rotation=45, ha="right", fontsize=fontsize)
ax["mainplot_ax"].set_xlabel("Gene", fontsize=fontsize)

ax["mainplot_ax"].set_ylabel("Leiden cluster", fontsize=fontsize)

ax["mainplot_ax"].tick_params(axis="both", labelsize=fontsize)

plt.tight_layout()

# # Save plot
# plt.savefig(f"host_leiden_marker-gene-expression.png", dpi=300, bbox_inches="tight")

Define celltypes:

In [ ]:
df_celltypes = pd.DataFrame()
df_celltypes["cluster"] = np.arange(19)
df_celltypes["celltype"] = [
    "B cells 4",
    "T cells 2",
    "B cells 6",
    "Natural killer 1",
    "B cells 2",
    "T cells 4",
    "T cells 5",
    "Monocytes 2",
    "Immature neutrophils 1",
    "T cells 1",
    "Monocytes 1",
    "B cells 1",
    "T cells 3",
    "Undefined 2",
    "B cells 5",
    "Undefined 3",
    "B cells 7",
    "B cells 3",
    "Undefined 1",
]

df_celltypes

In [ ]:
# Create copy of leiden cluster column to store manually assigned celltypes
host_adata.obs["celltype_clusters"] = host_adata.obs.leiden.copy()

# Relabel the clusters in celltype
host_adata.rename_categories("celltype_clusters", df_celltypes["celltype"].values)

# Sort celltype labels using categorical data type
celltype_order = [
    "B cells 1",
    "B cells 2",
    "B cells 3",
    "B cells 4",
    "B cells 5",
    "B cells 6",
    "B cells 7",
    "T cells 1",
    "T cells 2",
    "T cells 3",
    "T cells 4",
    "T cells 5",
    "Natural killer 1",
    "Monocytes 1",
    "Monocytes 2",
    "Immature neutrophils 1",
    "Undefined 1",
    "Undefined 2",
    "Undefined 3"
]
host_adata.obs["celltype_clusters"] = host_adata.obs["celltype_clusters"].cat.reorder_categories(list(celltype_order))

Create merged celltype column:

In [ ]:
host_adata.obs["celltype"] = [" ".join(i.split(" ")[:-1]) for i in host_adata.obs["celltype_clusters"]]

Merge timpeoints 7d and 8d due to few counts (also done in the original paper):

In [ ]:
new_tps = []
for tp in host_adata.obs["dpi_clean"].values:
    if tp == "7d" or tp == "8d":
        new_tps.append("7d/8d")
    else:
        new_tps.append(tp)

host_adata.obs["dpi_clean_merged"] = new_tps

___
Save host count matrix with celltype assignments:

In [ ]:
host_adata.write("macaque_QC_norm_leiden_celltypes.h5ad")